In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import datasets
from datasets import load_dataset, load_metric
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [14]:
# training info
training_dataset = "peixian/equity_evaluation_corpus"
training_dataset_split = "train"

train_labels_column = "gender"
train_features_column = "sentence" # the column tokenized for eval

num_labels = 2

relabel_training = None

def relabel_training(gender):
    return 0 if gender == "male" else 1

In [3]:
# evaluate info
evaluate_dataset = "social_bias_frames"
evaluate_input_column = 'post' # the column tokenized for eval

In [22]:
# trainer
model_checkpoint = "bert-base-cased"
batch_size = 8
metric=load_metric('bertscore')
trials = 1

In [5]:
dataset = load_dataset(training_dataset, split=training_dataset_split)

Reusing dataset equity_evaluation_corpus (/home/pw1329/.cache/huggingface/datasets/equity_evaluation_corpus/first_domain/1.1.0/697923ba8aa88f727772216e9fdc5fe17f04e9ea530aa691c0fed82207aa4c1a)


In [10]:
dataset.features

{'sentence': Value(dtype='string', id=None),
 'template': Value(dtype='string', id=None),
 'person': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'race': Value(dtype='string', id=None),
 'emotion': Value(dtype='string', id=None),
 'emotion word': Value(dtype='string', id=None)}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [12]:
d = dataset.train_test_split(test_size=0.1)

In [15]:
d = d.map(lambda x: {'labels': relabel_training(x[train_labels_column])})

In [16]:
d = d.map(lambda x: tokenizer(x[train_features_column], truncation=True, padding=True), batched=True)

In [17]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-bert",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir="./logs"
)

In [18]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }


In [19]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [20]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=d['train'],
    eval_dataset=d['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=trials, direction="maximize")

[I 2021-03-30 21:41:58,349] A new study created in memory with name: no-name-a24752ca-0825-427e-a980-890b4dbdb735
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
# load sbf

sbf = load_dataset(evaluate_dataset)

In [ ]:
sbf = sbf.map(lambda x: tokenizer(x[evaluate_input_column]), batched=True)

In [ ]:
type(sbf['train'])

In [ ]:
res = trainer.predict(sbf['validation'])

In [ ]:
res[0][:20]

In [ ]:
#uncomment to see posts
#sbf['train']['post'][:20]

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs/Mar28_22-25-53_gr004.nyu.cluster